This IS a Test of the installation

In [19]:
import sys, torch, tensorflow as tf, onnxruntime as ort, numpy as np
import onnx
from onnx import helper, TensorProto, save

# --- Versions Python & ONNX ---
print("Python:", sys.version.split()[0])
print("ONNX Version:", onnx.__version__)

# --- PyTorch ---
print("\n--- PyTorch ---")
print("Version:", torch.__version__)
print("CUDA disponible:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("GPU détecté:", torch.cuda.get_device_name(0))
    x = torch.randn(4096, 4096, device="cuda")
    y = x @ x
    print("Test multiplication matrice PyTorch OK, somme:", y.sum().item())

# --- TensorFlow ---
print("\n--- TensorFlow ---")
print("Version:", tf.__version__)
gpus = tf.config.list_physical_devices('GPU')
print("GPU TensorFlow détecté:", gpus)
if gpus:
    a = tf.random.normal([4096,4096])
    b = tf.random.normal([4096,4096])
    c = tf.matmul(a,b)
    print("Test multiplication matrice TF OK, somme:", tf.reduce_sum(c).numpy())

# --- ONNX Runtime ---
print("\n--- ONNX Runtime ---")
print("Version:", ort.__version__)
print("Providers disponibles:", ort.get_available_providers())

# --- Test ONNX 'Where' operator ---
print("\n--- Test ONNX 'Where' operator ---")
C = helper.make_tensor_value_info("C", TensorProto.BOOL, [3])
X = helper.make_tensor_value_info("X", TensorProto.FLOAT, [3])
Y = helper.make_tensor_value_info("Y", TensorProto.FLOAT, [3])
Z = helper.make_tensor_value_info("Z", TensorProto.FLOAT, [3])
node = helper.make_node("Where", ["C","X","Y"], ["Z"])
graph = helper.make_graph([node], "where_graph", [C,X,Y], [Z])
model = helper.make_model(graph, opset_imports=[helper.make_opsetid("", 23)])
save(model, "where_test.onnx")

for provider in ort.get_available_providers():
    try:
        sess = ort.InferenceSession("where_test.onnx", providers=[provider])
        cond = np.array([True, False, True], dtype=bool)
        x = np.array([1.0,2.0,3.0], dtype=np.float32)
        y = np.array([10.0,20.0,30.0], dtype=np.float32)
        z = sess.run(None, {"C":cond, "X":x, "Y":y})[0]
        print(f"Provider {provider}: ✅ Test Where OK, sortie:", z)
    except Exception as e:
        print(f"Provider {provider}: ❌ Erreur:", e)

# --- Comparaison NumPy ---
print("\n--- Comparaison NumPy ---")
cond = np.array([True, False, True])
x = np.array([1.0,2.0,3.0])
y = np.array([10.0,20.0,30.0])
z_np = np.where(cond,x,y)
print("Résultat NumPy:", z_np)

Python: 3.12.12
ONNX Version: 1.17.0

--- PyTorch ---
Version: 2.9.1+cu128
CUDA disponible: True
GPU détecté: NVIDIA GeForce RTX 3080 Laptop GPU
Test multiplication matrice PyTorch OK, somme: -164660.28125

--- TensorFlow ---
Version: 2.20.0
GPU TensorFlow détecté: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
Test multiplication matrice TF OK, somme: -4917.59

--- ONNX Runtime ---
Version: 1.23.2
Providers disponibles: ['TensorrtExecutionProvider', 'CUDAExecutionProvider', 'CPUExecutionProvider']

--- Test ONNX 'Where' operator ---
Provider TensorrtExecutionProvider: ✅ Test Where OK, sortie: [ 1. 20.  3.]
Provider CUDAExecutionProvider: ✅ Test Where OK, sortie: [ 1. 20.  3.]
Provider CPUExecutionProvider: ✅ Test Where OK, sortie: [ 1. 20.  3.]

--- Comparaison NumPy ---
Résultat NumPy: [ 1. 20.  3.]


In [20]:
import numpy as np
import onnxruntime as ort
import onnx
from onnx import helper, TensorProto, save

# --- Test ONNX 'MaxPool' operator Nominal ---
print("\n--- Test ONNX 'MaxPool' operator Nominal ---")
print("\n--- Rien de special a voir tout est OK, avec CPU/CUDA meme resultat ---")

onnx_type = TensorProto.UINT8

# Input / Output shapes
X = helper.make_tensor_value_info("X", onnx_type, [1, 1, 3, 3])
Y = helper.make_tensor_value_info("Y", onnx_type, [1, 1, 2, 2])
Indices = helper.make_tensor_value_info("Indices", TensorProto.INT64, [1, 1, 2, 2])

# MaxPool node (avec 2 sorties)
node = helper.make_node(
    "MaxPool",
    inputs=["X"],
    outputs=["Y", "Indices"],
    kernel_shape=[2, 2],
    pads=[0, 0, 0, 0],
    strides=[1, 1],
    dilations=[1, 1],
    ceil_mode=0
)

graph = helper.make_graph([node], "maxpool_graph_3x3", [X], [Y, Indices])
model = helper.make_model(graph, opset_imports=[helper.make_opsetid("", 22)])
save(model, "maxpool_test.onnx")

# Données demandées
data = [[[[ 0, 0, 5],
          [ 0, 0, 6],
          [ 7, 8, 0]]]]

x = np.array(data, dtype=np.uint8)

# Test pour tous les providers
for provider in ort.get_available_providers():
    try:
        sess = ort.InferenceSession("maxpool_test.onnx", providers=[provider])

        outputs = sess.run(None, {"X": x})
        y = outputs[0]
        indices = outputs[1]

        print(f"\nProvider {provider}: ✅ Test MaxPool OK")
        print("Sortie Y =\n", y)
        print("Indices =\n", indices)

    except Exception as e:
        print(f"\nProvider {provider}: ❌ Erreur:", e)



--- Test ONNX 'MaxPool' operator Nominal ---

--- Rien de special a voir tout est OK, avec CPU/CUDA meme resultat ---

Provider TensorrtExecutionProvider: ✅ Test MaxPool OK
Sortie Y =
 [[[[0 6]
   [8 8]]]]
Indices =
 [[[[-4  5]
   [ 7  7]]]]

Provider CUDAExecutionProvider: ✅ Test MaxPool OK
Sortie Y =
 [[[[0 6]
   [8 8]]]]
Indices =
 [[[[-4  5]
   [ 7  7]]]]

Provider CPUExecutionProvider: ✅ Test MaxPool OK
Sortie Y =
 [[[[0 6]
   [8 8]]]]
Indices =
 [[[[-4  5]
   [ 7  7]]]]


2026-02-16 15:51:42.731146135 [E:onnxruntime:Default, tensorrt_execution_provider.h:90 log] [2026-02-16 14:51:42   ERROR] ModelImporter.cpp:135: While parsing node number 0 [MaxPool -> "Y"]:
2026-02-16 15:51:42.731225035 [E:onnxruntime:Default, tensorrt_execution_provider.h:90 log] [2026-02-16 14:51:42   ERROR] ModelImporter.cpp:138: --- Begin node ---
input: "X"
output: "Y"
output: "Indices"
name: ""
op_type: "MaxPool"
attribute {
  name: "pads"
  ints: 0
  ints: 0
  ints: 0
  ints: 0
  type: INTS
}
attribute {
  name: "kernel_shape"
  ints: 2
  ints: 2
  type: INTS
}
attribute {
  name: "ceil_mode"
  i: 0
  type: INT
}
attribute {
  name: "strides"
  ints: 1
  ints: 1
  type: INTS
}
attribute {
  name: "storage_order"
  i: 0
  type: INT
}
attribute {
  name: "dilations"
  ints: 1
  ints: 1
  type: INTS
}
attribute {
  name: "auto_pad"
  s: "NOTSET"
  type: STRING
}

2026-02-16 15:51:42.731256038 [E:onnxruntime:Default, tensorrt_execution_provider.h:90 log] [2026-02-16 14:51:42   ERRO

In [21]:
import numpy as np
import onnxruntime as ort
import onnx
from onnx import helper, TensorProto, save

# --- Test ONNX 'MaxPool' operator (DOUBLE + padding + indices) ---
print("\n--- Test ONNX 'MaxPool' operator (DOUBLE + padding + indices) ---")
print("\n--- voir le -4 (quand padding), avec CPU/CUDA meme resultat ---")

onnx_type = TensorProto.DOUBLE

# Input / Output shapes
X = helper.make_tensor_value_info("X", onnx_type, [1, 1, 3, 3])
Y = helper.make_tensor_value_info("Y", onnx_type, [1, 1, 4, 4])
Indices = helper.make_tensor_value_info("Indices", TensorProto.INT64, [1, 1, 4, 4])

# MaxPool node
node = helper.make_node(
    "MaxPool",
    inputs=["X"],
    outputs=["Y", "Indices"],
    kernel_shape=[2, 2],
    pads=[1, 1, 1, 1],
    strides=[1, 1],
    dilations=[1, 1],
    ceil_mode=0
)

graph = helper.make_graph([node], "maxpool_graph_double_pad", [X], [Y, Indices])
model = helper.make_model(graph, opset_imports=[helper.make_opsetid("", 22)])
save(model, "maxpool_test.onnx")

# Données demandées
data = [[[[0.0, 9.57875561, 4.56432533],
          [2.72844928, 3.54234851, 2.55354471],
          [2.83691720, 3.46789489, -np.inf]]]]

x = np.array(data, dtype=np.float64)

# Test pour tous les providers
for provider in ort.get_available_providers():
    try:
        sess = ort.InferenceSession("maxpool_test.onnx", providers=[provider])

        outputs = sess.run(None, {"X": x})
        y = outputs[0]
        indices = outputs[1]

        print(f"\nProvider {provider}: ✅ Test MaxPool OK")
        print("Sortie Y =\n", y)
        print("Indices =\n", indices)

    except Exception as e:
        print(f"\nProvider {provider}: ❌ Erreur:", e)



--- Test ONNX 'MaxPool' operator (DOUBLE + padding + indices) ---

--- voir le -4 (quand padding), avec CPU/CUDA meme resultat ---

Provider TensorrtExecutionProvider: ✅ Test MaxPool OK
Sortie Y =
 [[[[ 0.00000000e+000  9.57875561e+000  9.57875561e+000  4.56432533e+000]
   [ 2.72844928e+000  9.57875561e+000  9.57875561e+000  4.56432533e+000]
   [ 2.83691720e+000  3.54234851e+000  3.54234851e+000  2.55354471e+000]
   [ 2.83691720e+000  3.46789489e+000  3.46789489e+000 -1.79769313e+308]]]]
Indices =
 [[[[ 0  1  1  2]
   [ 3  1  1  2]
   [ 6  4  4  5]
   [ 6  7  7 -4]]]]

Provider CUDAExecutionProvider: ✅ Test MaxPool OK
Sortie Y =
 [[[[ 0.00000000e+000  9.57875561e+000  9.57875561e+000  4.56432533e+000]
   [ 2.72844928e+000  9.57875561e+000  9.57875561e+000  4.56432533e+000]
   [ 2.83691720e+000  3.54234851e+000  3.54234851e+000  2.55354471e+000]
   [ 2.83691720e+000  3.46789489e+000  3.46789489e+000 -1.79769313e+308]]]]
Indices =
 [[[[ 0  1  1  2]
   [ 3  1  1  2]
   [ 6  4  4  5]
   [

2026-02-16 15:51:42.757909700 [E:onnxruntime:Default, tensorrt_execution_provider.h:90 log] [2026-02-16 14:51:42   ERROR] ModelImporter.cpp:135: While parsing node number 0 [MaxPool -> "Y"]:
2026-02-16 15:51:42.757951965 [E:onnxruntime:Default, tensorrt_execution_provider.h:90 log] [2026-02-16 14:51:42   ERROR] ModelImporter.cpp:138: --- Begin node ---
input: "X"
output: "Y"
output: "Indices"
name: ""
op_type: "MaxPool"
attribute {
  name: "pads"
  ints: 1
  ints: 1
  ints: 1
  ints: 1
  type: INTS
}
attribute {
  name: "kernel_shape"
  ints: 2
  ints: 2
  type: INTS
}
attribute {
  name: "ceil_mode"
  i: 0
  type: INT
}
attribute {
  name: "strides"
  ints: 1
  ints: 1
  type: INTS
}
attribute {
  name: "storage_order"
  i: 0
  type: INT
}
attribute {
  name: "dilations"
  ints: 1
  ints: 1
  type: INTS
}
attribute {
  name: "auto_pad"
  s: "NOTSET"
  type: STRING
}

2026-02-16 15:51:42.757957408 [E:onnxruntime:Default, tensorrt_execution_provider.h:90 log] [2026-02-16 14:51:42   ERRO

In [22]:
import numpy as np
import onnx
from onnx import helper, TensorProto, save
import onnxruntime as ort

print("\n--- Test ONNX 'MaxPool' operator pour obtenir indice -3 ---")
print("\n--- voir le -4 (quand padding), avec CPU/CUDA mais divergence cette fois ---")

onnx_type = TensorProto.FLOAT

# Input / Output shapes
X = helper.make_tensor_value_info("X", onnx_type, [1, 1, 2, 2])
Y = helper.make_tensor_value_info("Y", onnx_type, [1, 1, 3, 3])
Indices = helper.make_tensor_value_info("Indices", TensorProto.INT64, [1, 1, 3, 3])

# MaxPool node (2x2 kernel, stride 1, padding 1)
node = helper.make_node(
    "MaxPool",
    inputs=["X"],
    outputs=["Y", "Indices"],
    kernel_shape=[2, 2],
    pads=[1, 1, 1, 1],
    strides=[1, 1],
    dilations=[1, 1],
    ceil_mode=0
)

graph = helper.make_graph([node], "maxpool_graph_neg3", [X], [Y, Indices])
model = helper.make_model(graph, opset_imports=[helper.make_opsetid("", 13)])
save(model, "maxpool_neg3.onnx")

# Données d'entrée (2x2)
data = [[[[0.0, -np.inf],  # placer -inf dans un coin pour forcer un indice négatif
          [1.0, 1.1]]]]

x = np.array(data, dtype=np.float32)

# Test pour tous les providers disponibles
for provider in ort.get_available_providers():
    try:
        sess = ort.InferenceSession("maxpool_neg3.onnx", providers=[provider])

        outputs = sess.run(None, {"X": x})
        y = outputs[0]
        indices = outputs[1]

        print(f"\nProvider {provider}: ✅ Test MaxPool OK")
        print("Sortie Y =\n", y)
        print("Indices =\n", indices)

    except Exception as e:
        print(f"\nProvider {provider}: ❌ Erreur:", e)



--- Test ONNX 'MaxPool' operator pour obtenir indice -3 ---

--- voir le -4 (quand padding), avec CPU/CUDA mais divergence cette fois ---

Provider TensorrtExecutionProvider: ✅ Test MaxPool OK
Sortie Y =
 [[[[ 0.0000000e+00  0.0000000e+00 -3.4028235e+38]
   [ 1.0000000e+00  1.1000000e+00  1.1000000e+00]
   [ 1.0000000e+00  1.1000000e+00  1.1000000e+00]]]]
Indices =
 [[[[ 0  0 -3]
   [ 2  3  3]
   [ 2  3  3]]]]

Provider CUDAExecutionProvider: ✅ Test MaxPool OK
Sortie Y =
 [[[[0.  0.  0. ]
   [1.  1.1 1.1]
   [1.  1.1 1.1]]]]
Indices =
 [[[[ 0  0 -4]
   [ 2  3  3]
   [ 2  3  3]]]]

Provider CPUExecutionProvider: ✅ Test MaxPool OK
Sortie Y =
 [[[[ 0.0000000e+00  0.0000000e+00 -3.4028235e+38]
   [ 1.0000000e+00  1.1000000e+00  1.1000000e+00]
   [ 1.0000000e+00  1.1000000e+00  1.1000000e+00]]]]
Indices =
 [[[[ 0  0 -3]
   [ 2  3  3]
   [ 2  3  3]]]]


2026-02-16 15:51:42.789165513 [E:onnxruntime:Default, tensorrt_execution_provider.h:90 log] [2026-02-16 14:51:42   ERROR] ModelImporter.cpp:135: While parsing node number 0 [MaxPool -> "Y"]:
2026-02-16 15:51:42.789208630 [E:onnxruntime:Default, tensorrt_execution_provider.h:90 log] [2026-02-16 14:51:42   ERROR] ModelImporter.cpp:138: --- Begin node ---
input: "X"
output: "Y"
output: "Indices"
name: ""
op_type: "MaxPool"
attribute {
  name: "pads"
  ints: 1
  ints: 1
  ints: 1
  ints: 1
  type: INTS
}
attribute {
  name: "kernel_shape"
  ints: 2
  ints: 2
  type: INTS
}
attribute {
  name: "ceil_mode"
  i: 0
  type: INT
}
attribute {
  name: "strides"
  ints: 1
  ints: 1
  type: INTS
}
attribute {
  name: "storage_order"
  i: 0
  type: INT
}
attribute {
  name: "dilations"
  ints: 1
  ints: 1
  type: INTS
}
attribute {
  name: "auto_pad"
  s: "NOTSET"
  type: STRING
}

2026-02-16 15:51:42.789213502 [E:onnxruntime:Default, tensorrt_execution_provider.h:90 log] [2026-02-16 14:51:42   ERRO

In [23]:
import numpy as np
import onnx
from onnx import helper, TensorProto, save
import onnxruntime as ort

print("\n--- Test ONNX 'MaxPool' operator ---")
print("\n--- voir le -5 (quand padding), avec CPU/CUDA mais divergence cette fois ---")

onnx_type = TensorProto.FLOAT

# Input / Output shapes
# On met une dimension H=3, W=4 pour matcher les données
X = helper.make_tensor_value_info("X", onnx_type, [1, 1, 3, 4])
# On met des dimensions dynamiques pour laisser ONNX calculer la sortie
Y = helper.make_tensor_value_info("Y", onnx_type, [1, 1, 3, 4])
Indices = helper.make_tensor_value_info("Indices", TensorProto.INT64, [1, 1, 3, 4])

# MaxPool node (2x2 kernel, stride 1, padding 1)
node = helper.make_node(
    "MaxPool",
    inputs=["X"],
    outputs=["Y", "Indices"],
    kernel_shape=[2, 2],
    pads=[1, 1, 1, 1],
    strides=[1, 1],
    ceil_mode=0
)

graph = helper.make_graph([node], "maxpool_graph_corrige", [X], [Y, Indices])
model = helper.make_model(graph, opset_imports=[helper.make_opsetid("", 13)])
save(model, "maxpool_corrige.onnx")

# Données d'entrée (1,1,3,4)
data = [[[[ -np.inf, 0.1, 0.2, 0.3],
           [ 1.0, 1.1, 1.2, 1.3],
           [ 2.0, 2.1, 2.2, 2.3]]]]

x = np.array(data, dtype=np.float32)

# Test pour tous les providers disponibles
for provider in ort.get_available_providers():
    try:
        sess = ort.InferenceSession("maxpool_corrige.onnx", providers=[provider])

        outputs = sess.run(None, {"X": x})
        y = outputs[0]
        indices = outputs[1]

        print(f"\nProvider {provider}: ✅ Test MaxPool OK")
        print("Sortie Y =\n", y)
        print("Indices =\n", indices)

    except Exception as e:
        print(f"\nProvider {provider}: ❌ Erreur:", e)



--- Test ONNX 'MaxPool' operator ---

--- voir le -5 (quand padding), avec CPU/CUDA mais divergence cette fois ---

Provider TensorrtExecutionProvider: ✅ Test MaxPool OK
Sortie Y =
 [[[[-3.4028235e+38  1.0000000e-01  2.0000000e-01  3.0000001e-01
     3.0000001e-01]
   [ 1.0000000e+00  1.1000000e+00  1.2000000e+00  1.3000000e+00
     1.3000000e+00]
   [ 2.0000000e+00  2.0999999e+00  2.2000000e+00  2.3000000e+00
     2.3000000e+00]
   [ 2.0000000e+00  2.0999999e+00  2.2000000e+00  2.3000000e+00
     2.3000000e+00]]]]
Indices =
 [[[[-5  1  2  3  3]
   [ 4  5  6  7  7]
   [ 8  9 10 11 11]
   [ 8  9 10 11 11]]]]

Provider CUDAExecutionProvider: ✅ Test MaxPool OK
Sortie Y =
 [[[[0.  0.1 0.2 0.3 0.3]
   [1.  1.1 1.2 1.3 1.3]
   [2.  2.1 2.2 2.3 2.3]
   [2.  2.1 2.2 2.3 2.3]]]]
Indices =
 [[[[-6  1  2  3  3]
   [ 4  5  6  7  7]
   [ 8  9 10 11 11]
   [ 8  9 10 11 11]]]]

Provider CPUExecutionProvider: ✅ Test MaxPool OK
Sortie Y =
 [[[[-3.4028235e+38  1.0000000e-01  2.0000000e-01  3.0000001e-0

2026-02-16 15:51:42.818010512 [W:onnxruntime:, graph.cc:120 MergeShapeInfo] Error merging shape info for output. 'Y' source:{1,1,4,5} target:{1,1,3,4}. Falling back to lenient merge.
2026-02-16 15:51:42.818032198 [W:onnxruntime:, graph.cc:120 MergeShapeInfo] Error merging shape info for output. 'Indices' source:{1,1,4,5} target:{1,1,3,4}. Falling back to lenient merge.
2026-02-16 15:51:42.820036971 [E:onnxruntime:Default, tensorrt_execution_provider.h:90 log] [2026-02-16 14:51:42   ERROR] ModelImporter.cpp:135: While parsing node number 0 [MaxPool -> "Y"]:
2026-02-16 15:51:42.820072954 [E:onnxruntime:Default, tensorrt_execution_provider.h:90 log] [2026-02-16 14:51:42   ERROR] ModelImporter.cpp:138: --- Begin node ---
input: "X"
output: "Y"
output: "Indices"
name: ""
op_type: "MaxPool"
attribute {
  name: "storage_order"
  i: 0
  type: INT
}
attribute {
  name: "strides"
  ints: 1
  ints: 1
  type: INTS
}
attribute {
  name: "pads"
  ints: 1
  ints: 1
  ints: 1
  ints: 1
  type: INTS
}


In [24]:
import numpy as np
import onnx
from onnx import helper, TensorProto, save
import onnxruntime as ort

print("\n--- Test ONNX 'MaxPool' triple divergence indice et valeur ---")
print("\n--- voir le -3/-4 et le 3/-4 (quand padding), avec CPU/CUDA double divergence cette fois ---")
print("\n--- voir valeur de Y qui diverge aussi inf vs 0 ---")

onnx_type = TensorProto.FLOAT

# Input / Output shapes
X = helper.make_tensor_value_info("X", onnx_type, [1, 1, 2, 2])
Y = helper.make_tensor_value_info("Y", onnx_type, [1, 1, 3, 3])
Indices = helper.make_tensor_value_info("Indices", TensorProto.INT64, [1, 1, 3, 3])

# MaxPool node (2x2 kernel, stride 1, padding 1)
node = helper.make_node(
    "MaxPool",
    inputs=["X"],
    outputs=["Y", "Indices"],
    kernel_shape=[2, 2],
    pads=[1, 1, 1, 1],
    strides=[1, 1],
    dilations=[1, 1],
    ceil_mode=0
)

graph = helper.make_graph([node], "maxpool_graph_neg3", [X], [Y, Indices])
model = helper.make_model(graph, opset_imports=[helper.make_opsetid("", 13)])
save(model, "maxpool_neg3.onnx")

# Données d'entrée (2x2)
data = [[[[np.nan, np.nan],  # placer -inf dans un coin pour forcer un indice négatif
          [np.nan, 1.1]]]]

x = np.array(data, dtype=np.float32)

# Test pour tous les providers disponibles
for provider in ort.get_available_providers():
    try:
        sess = ort.InferenceSession("maxpool_neg3.onnx", providers=[provider])

        outputs = sess.run(None, {"X": x})
        y = outputs[0]
        indices = outputs[1]

        print(f"\nProvider {provider}: ✅ Test MaxPool OK")
        print("Sortie Y =\n", y)
        print("Indices =\n", indices)

    except Exception as e:
        print(f"\nProvider {provider}: ❌ Erreur:", e)



--- Test ONNX 'MaxPool' triple divergence indice et valeur ---

--- voir le -3/-4 et le 3/-4 (quand padding), avec CPU/CUDA double divergence cette fois ---

--- voir valeur de Y qui diverge aussi inf vs 0 ---

Provider TensorrtExecutionProvider: ✅ Test MaxPool OK
Sortie Y =
 [[[[-3.4028235e+38 -3.4028235e+38 -3.4028235e+38]
   [-3.4028235e+38  1.1000000e+00  1.1000000e+00]
   [-3.4028235e+38  1.1000000e+00  1.1000000e+00]]]]
Indices =
 [[[[-3 -3 -3]
   [-3  3  3]
   [-3  3  3]]]]

Provider CUDAExecutionProvider: ✅ Test MaxPool OK
Sortie Y =
 [[[[0.  0.  0. ]
   [0.  0.  0. ]
   [0.  0.  1.1]]]]
Indices =
 [[[[-4 -4 -4]
   [-4 -4 -4]
   [-4 -4  3]]]]

Provider CPUExecutionProvider: ✅ Test MaxPool OK
Sortie Y =
 [[[[-3.4028235e+38 -3.4028235e+38 -3.4028235e+38]
   [-3.4028235e+38  1.1000000e+00  1.1000000e+00]
   [-3.4028235e+38  1.1000000e+00  1.1000000e+00]]]]
Indices =
 [[[[-3 -3 -3]
   [-3  3  3]
   [-3  3  3]]]]


2026-02-16 15:51:42.846049392 [E:onnxruntime:Default, tensorrt_execution_provider.h:90 log] [2026-02-16 14:51:42   ERROR] ModelImporter.cpp:135: While parsing node number 0 [MaxPool -> "Y"]:
2026-02-16 15:51:42.846090365 [E:onnxruntime:Default, tensorrt_execution_provider.h:90 log] [2026-02-16 14:51:42   ERROR] ModelImporter.cpp:138: --- Begin node ---
input: "X"
output: "Y"
output: "Indices"
name: ""
op_type: "MaxPool"
attribute {
  name: "pads"
  ints: 1
  ints: 1
  ints: 1
  ints: 1
  type: INTS
}
attribute {
  name: "kernel_shape"
  ints: 2
  ints: 2
  type: INTS
}
attribute {
  name: "ceil_mode"
  i: 0
  type: INT
}
attribute {
  name: "strides"
  ints: 1
  ints: 1
  type: INTS
}
attribute {
  name: "storage_order"
  i: 0
  type: INT
}
attribute {
  name: "dilations"
  ints: 1
  ints: 1
  type: INTS
}
attribute {
  name: "auto_pad"
  s: "NOTSET"
  type: STRING
}

2026-02-16 15:51:42.846095006 [E:onnxruntime:Default, tensorrt_execution_provider.h:90 log] [2026-02-16 14:51:42   ERRO

In [25]:
import numpy as np
import onnx
from onnx import helper, TensorProto, save
import onnxruntime as ort

print("\n--- Test ONNX 'MaxPool' operator  ---")
print("\n--- voir le valeur et indice avec CPU/CUDA double divergence autre exemple ---")

onnx_type = TensorProto.FLOAT

# Input / Output shapes
# On met une dimension H=3, W=4 pour matcher les données
X = helper.make_tensor_value_info("X", onnx_type, [1, 1, 3, 4])
# On met des dimensions dynamiques pour laisser ONNX calculer la sortie
Y = helper.make_tensor_value_info("Y", onnx_type, [1, 1, 3, 4])
Indices = helper.make_tensor_value_info("Indices", TensorProto.INT64, [1, 1, 3, 4])

# MaxPool node (2x2 kernel, stride 1, padding 1)
node = helper.make_node(
    "MaxPool",
    inputs=["X"],
    outputs=["Y", "Indices"],
    kernel_shape=[2, 2],
    pads=[1, 1, 1, 1],
    strides=[1, 1],
    ceil_mode=0
)

graph = helper.make_graph([node], "maxpool_graph_corrige", [X], [Y, Indices])
model = helper.make_model(graph, opset_imports=[helper.make_opsetid("", 13)])
save(model, "maxpool_corrige.onnx")

# Données d'entrée (1,1,3,4)
data = [[[[ np.nan, np.nan, np.nan, np.nan],
           [ np.nan, 0.0, np.nan, np.nan],
           [ np.nan, np.nan, np.nan, np.nan]]]]

x = np.array(data, dtype=np.float32)

# Test pour tous les providers disponibles
for provider in ort.get_available_providers():
    try:
        sess = ort.InferenceSession("maxpool_corrige.onnx", providers=[provider])

        outputs = sess.run(None, {"X": x})
        y = outputs[0]
        indices = outputs[1]

        print(f"\nProvider {provider}: ✅ Test MaxPool OK")
        print("Sortie Y =\n", y)
        print("Indices =\n", indices)

    except Exception as e:
        print(f"\nProvider {provider}: ❌ Erreur:", e)



--- Test ONNX 'MaxPool' operator  ---

--- voir le valeur et indice avec CPU/CUDA double divergence autre exemple ---

Provider TensorrtExecutionProvider: ✅ Test MaxPool OK
Sortie Y =
 [[[[-3.4028235e+38 -3.4028235e+38 -3.4028235e+38 -3.4028235e+38
    -3.4028235e+38]
   [-3.4028235e+38  0.0000000e+00  0.0000000e+00 -3.4028235e+38
    -3.4028235e+38]
   [-3.4028235e+38  0.0000000e+00  0.0000000e+00 -3.4028235e+38
    -3.4028235e+38]
   [-3.4028235e+38 -3.4028235e+38 -3.4028235e+38 -3.4028235e+38
    -3.4028235e+38]]]]
Indices =
 [[[[-5 -5 -5 -5 -5]
   [-5  5  5 -5 -5]
   [-5  5  5 -5 -5]
   [-5 -5 -5 -5 -5]]]]

Provider CUDAExecutionProvider: ✅ Test MaxPool OK
Sortie Y =
 [[[[0. 0. 0. 0. 0.]
   [0. 0. 0. 0. 0.]
   [0. 0. 0. 0. 0.]
   [0. 0. 0. 0. 0.]]]]
Indices =
 [[[[-6 -6 -6 -6 -6]
   [-6 -6 -6 -6 -6]
   [-6 -6  5 -6 -6]
   [-6 -6 -6 -6 -6]]]]

Provider CPUExecutionProvider: ✅ Test MaxPool OK
Sortie Y =
 [[[[-3.4028235e+38 -3.4028235e+38 -3.4028235e+38 -3.4028235e+38
    -3.4028235e

2026-02-16 15:51:42.871647746 [W:onnxruntime:, graph.cc:120 MergeShapeInfo] Error merging shape info for output. 'Y' source:{1,1,4,5} target:{1,1,3,4}. Falling back to lenient merge.
2026-02-16 15:51:42.871670263 [W:onnxruntime:, graph.cc:120 MergeShapeInfo] Error merging shape info for output. 'Indices' source:{1,1,4,5} target:{1,1,3,4}. Falling back to lenient merge.
2026-02-16 15:51:42.873849900 [E:onnxruntime:Default, tensorrt_execution_provider.h:90 log] [2026-02-16 14:51:42   ERROR] ModelImporter.cpp:135: While parsing node number 0 [MaxPool -> "Y"]:
2026-02-16 15:51:42.873885598 [E:onnxruntime:Default, tensorrt_execution_provider.h:90 log] [2026-02-16 14:51:42   ERROR] ModelImporter.cpp:138: --- Begin node ---
input: "X"
output: "Y"
output: "Indices"
name: ""
op_type: "MaxPool"
attribute {
  name: "storage_order"
  i: 0
  type: INT
}
attribute {
  name: "strides"
  ints: 1
  ints: 1
  type: INTS
}
attribute {
  name: "pads"
  ints: 1
  ints: 1
  ints: 1
  ints: 1
  type: INTS
}


In [26]:
import numpy as np
import onnx
from onnx import helper, TensorProto, save
import onnxruntime as ort
import os

print("\n=== Test minimal divergence NaN / Inf sur MaxPool ===")
print("\n--- comparer les resultats quand on appel l'operateur avec indice et sans indice, ca influence les valeur de sortie ---")
print("\n--- voir ce test et celui d'apres ---")


# ==========================================================
# 1️⃣ Modèle MaxPool simple
# ==========================================================

X = helper.make_tensor_value_info("X", TensorProto.FLOAT, [1, 1, 2, 2])
Y = helper.make_tensor_value_info("Y", TensorProto.FLOAT, [1, 1, 3, 3])

node = helper.make_node(
    "MaxPool",
    inputs=["X"],
    outputs=["Y"],
    kernel_shape=[2, 2],
    pads=[1, 1, 1, 1],
    strides=[1, 1]
)

graph = helper.make_graph([node], "maxpool_nan_inf", [X], [Y])
model = helper.make_model(graph, opset_imports=[helper.make_opsetid("", 13)])
save(model, "maxpool_nan_inf.onnx")

# ==========================================================
# 2️⃣ Input ciblé : NaN + +Inf
# ==========================================================

x = np.array(
    [[[[np.nan, np.inf],
       [-1.0, 1.0]]]],
    dtype=np.float32
)

print("\nEntrée X =")
print(x)

# ==========================================================
# 3️⃣ Exécution CPU vs CUDA
# ==========================================================

results = {}

for provider in ["CPUExecutionProvider", "CUDAExecutionProvider"]:
    if provider not in ort.get_available_providers():
        continue

    sess = ort.InferenceSession("maxpool_nan_inf.onnx", providers=[provider])
    y = sess.run(None, {"X": x})[0]
    results[provider] = y

    print(f"\nProvider {provider}")
    print("Sortie Y =")
    print(y)




=== Test minimal divergence NaN / Inf sur MaxPool ===

--- comparer les resultats quand on appel l'operateur avec indice et sans indice, ca influence les valeur de sortie ---

--- voir ce test et celui d'apres ---

Entrée X =
[[[[nan inf]
   [-1.  1.]]]]

Provider CPUExecutionProvider
Sortie Y =
[[[[nan inf inf]
   [nan inf inf]
   [-1.  1.  1.]]]]

Provider CUDAExecutionProvider
Sortie Y =
[[[[nan nan inf]
   [nan nan inf]
   [-1.  1.  1.]]]]


In [ ]:
import numpy as np
import onnx
from onnx import helper, TensorProto, save
import onnxruntime as ort
import os

print("\n=== Test minimal divergence NaN / Inf sur MaxPool (avec indices) ===")
print("\n--- comparer les resultats quand on appel l'operateur avec indice et sans indice, ca influence les valeur de sortie ---")
print("\n--- voir ce test et celui d'avant des nan et des inf deviennent egale a 0---")

# ==========================================================
# 1️⃣ Modèle MaxPool simple (retourne Y et Indices)
# ==========================================================

X = helper.make_tensor_value_info("X", TensorProto.FLOAT, [1, 1, 2, 2])
Y = helper.make_tensor_value_info("Y", TensorProto.FLOAT, [1, 1, 3, 3])
Indices = helper.make_tensor_value_info("Indices", TensorProto.INT64, [1, 1, 3, 3])

node = helper.make_node(
    "MaxPool",
    inputs=["X"],
    outputs=["Y", "Indices"],
    kernel_shape=[2, 2],
    pads=[1, 1, 1, 1],
    strides=[1, 1]
)

graph = helper.make_graph([node], "maxpool_nan_inf", [X], [Y, Indices])
model = helper.make_model(graph, opset_imports=[helper.make_opsetid("", 13)])
save(model, "maxpool_nan_inf.onnx")

# ==========================================================
# 2️⃣ Input ciblé : NaN + +Inf
# ==========================================================

x = np.array(
    [[[[np.nan, np.inf],
       [-1.0, 1.0]]]],
    dtype=np.float32
)

print("\nEntrée X =")
print(x)

# ==========================================================
# 3️⃣ Exécution CPU vs CUDA
# ==========================================================

results = {}

for provider in ["CPUExecutionProvider", "CUDAExecutionProvider"]:
    if provider not in ort.get_available_providers():
        continue

    sess = ort.InferenceSession("maxpool_nan_inf.onnx", providers=[provider])
    y, indices = sess.run(None, {"X": x})
    results[provider] = (y, indices)

    print(f"\nProvider {provider}")
    print("Sortie Y =")
    print(y)
    print("Indices =")
    print(indices)




=== Test minimal divergence NaN / Inf sur MaxPool (avec indices) ===

--- comparer les resultats quand on appel l'operateur avec indice et sans indice, ca influence les valeur de sortie ---

--- voir ce test et celui d'avant ---

Entrée X =
[[[[nan inf]
   [-1.  1.]]]]

Provider CPUExecutionProvider
Sortie Y =
[[[[-3.4028235e+38            inf            inf]
   [-1.0000000e+00            inf            inf]
   [-1.0000000e+00  1.0000000e+00  1.0000000e+00]]]]
Indices =
[[[[-3  1  1]
   [ 2  1  1]
   [ 2  3  3]]]]

Provider CUDAExecutionProvider
Sortie Y =
[[[[ 0.  0.  0.]
   [ 0.  0.  0.]
   [-1.  1.  1.]]]]
Indices =
[[[[-4 -4 -4]
   [-4 -4 -4]
   [ 2  3  3]]]]
